In [ ]:
import itertools as it
import operator
import re
from collections import Counter
from pathlib import Path

import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc
import pysam
from pyarrow import csv
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.io as sio

In [ ]:
hv.extension("bokeh")

In [ ]:
import pyinstrument

%load_ext pyinstrument
import line_profiler

%load_ext line_profiler

# Config

In [ ]:
data_dir = Path(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/"
)

# GAF to Parquet

In [ ]:
gaf_filenames = list(data_dir.glob("*_barcodeonly.gaf"))

In [ ]:
gaf_filenames

In [ ]:
gaf_filename = gaf_filenames[0]

In [ ]:
gaf_filename.lstat().st_size / 1e6

In [ ]:
%%time
gaf_table = pa.Table.from_batches(tqdm(sio.iter_gaf(gaf_filename)))

In [ ]:
gaf_table.nbytes / 1e6

In [ ]:
{n: gaf_table.column(n).nbytes // 1e6 for n in gaf_table.column_names}

# GAF/BAM to Parquet

In [ ]:
bam_filename = str(gaf_filename).replace("_barcodeonly.gaf", ".bam")

In [ ]:
bam = pysam.AlignmentFile(bam_filename, check_sq=False)

In [ ]:
read = next(bam.fetch(until_eof=True))
read.query_name

In [ ]:
bam.reset()

In [ ]:
bam_index = pysam.IndexedReads(bam)

In [ ]:
%%time
bam.reset()
bam_index.build()

In [ ]:
batch = next(sio.iter_bam_and_gaf(bam_filename, gaf_filename))

In [ ]:
{k: getattr(read, k) for k in dir(read) if not k.startswith("_")}

In [ ]:
read.query_sequence

In [ ]:
import array
import random

In [ ]:
x = array.array("B", [random.randint(0, 255) for i in range(50000)])

In [ ]:
%timeit pa.array(x, pa.uint8())

In [ ]:
%timeit pa.array(np.asarray(x), pa.uint8())

In [ ]:
read.query_qualities

In [ ]:
pa.array([read.query_qualities[:3], read.query_qualities[:5]], pa.list_(pa.uint8()))

## Benchmarking

In [ ]:
%pyinstrument list(it.islice(sio.iter_bam_and_gaf(bam, gaf_filename, bam_index=bam_index), 10))[:0]

In [ ]:
%lprun -f sio.iter_bam_and_gaf list(it.islice(sio.iter_bam_and_gaf(bam, gaf_filename, bam_index=bam_index), 10))[:0]

In [ ]:
x = read.query_qualities

In [ ]:
np.array(x)

In [ ]:
%lprun -f sio.iter_bam_and_gaf list(sio.iter_bam_and_gaf(bam_filename, gaf_filename))[:0]

In [ ]:
%pyinstrument list(sio.iter_bam_and_gaf(bam_filename, gaf_filename))[:0]

In [ ]:
%%time
table = pa.Table.from_batches(tqdm(sio.iter_bam_and_gaf(bam_filename, gaf_filename)))

In [ ]:
%%time
table2 = pa.Table.from_batches(
    tqdm(sio.iter_bam_and_gaf(bam, gaf_filename, bam_index=bam_index))
)

In [ ]:
x = list(
    tqdm(it.islice(sio.iter_bam_and_gaf(bam, gaf_filename, bam_index=bam_index), 2))
)

In [ ]:
x[0].column("name")

In [ ]:
len(x[0].column("name"))

In [ ]:
len(x[1].column("name"))

In [ ]:
table.nbytes / 1e6

In [ ]:
bam_filename

In [ ]:
len(table2)

In [ ]:
z = table2.column("name").value_counts()

In [ ]:
zz = z.sort("descending", "counts")

In [ ]:
zzz = pc.filter(zz.field("values"), pc.not_equal(zz.field("counts"), 1))

In [ ]:
zzz

In [ ]:
pc.index_in(zzz, table2.column("name"))

In [ ]:
zz.field("values")[zz.field("counts") != 1]

In [ ]:
len(table2) - len(pc.unique(table2.column("name")))

In [ ]:
%%time
len(pc.unique(table2.column("name")))

In [ ]:
%%time
len(set(table.column("name")))

In [ ]:
%%time
bam.reset()
reads = 0
for read in bam.fetch(until_eof=True):
    reads += 1

In [ ]:
%%time
bam.reset()
reads = set(read.query_name for read in tqdm(bam.fetch(until_eof=True)))

In [ ]:
%%time
bam.reset()
reads = 0
names = []
for read in it.islice(bam.fetch(until_eof=True), 10):
    names.append(read.query_name)
    reads += 1
for read in it.islice(bam.fetch(until_eof=True), 10):
    names.append(read.query_name)
    reads += 1

In [ ]:
names

In [ ]:
len(reads)

In [ ]:
len(table)